In [7]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import time
import math
import numpy as np
import random
import torch
from torch.nn.parameter import Parameter
from torch.utils.data import Dataset, DataLoader

In [49]:
#This class generates correlated memory vectors as decribed in (Benna, Fusi; 2021)
class CorrelatedPatterns():
    def __init__(self, 
                 L, #Length of each memory vector 
                 p, #Number of ancestors
                 k, #Number of children per ancestor
                 gamma): #Average overlap between child and ancestor. A value of one means each child is identical to its ancestor,
                        #while a value of zero means each child is completely different from its ancestor.
        self.L = L
        self.p = p
        self.k = k
        self.gamma = gamma
        
        #Create three arrays to store the ancestor vectors, the descendant (child) vectors, and the difference vectors
        self.ancestors = []
        self.descendants = []
        self.differences = []
        
        #For purposes of PyTorch dataset creation, we will create two new lists that do not themselves contain lists
        self.descendants_singlelist = []
        self.differences_singlelist = []
        
        for _ancestorIndex in range(p):
            
            #Each ancestor is initialized randomly
            ancestor = np.random.choice((0,1), size=(L))
            self.ancestors.append(np.array(ancestor))
            
            self.descendants.append([])
            #Initialize k descendants
            for _descendantIndex in range(k):
                descendant = torch.tensor([])
                for __i in range(len(ancestor)):
                    
                    #With probability 1-gamma, the descendant memory is corrupted at this bit. 
                    if(random.uniform(0,1) < 1-gamma):
                        descendant = torch.cat((descendant, torch.tensor([0 if random.uniform(0,1) < 0.5 else 1])))
                    else: #Otherwise, the ancestor's memory at this bit is copied to the descendant.
                        descendant = torch.cat((descendant, torch.tensor([ancestor[__i]])))
                
                #Save the memory
                self.descendants[_ancestorIndex].append(descendant.clone().detach())
                self.descendants_singlelist.append(descendant.clone().detach().reshape(1,-1))
            
            #Calculate the differences between the ancestor vectors and the child vectors
            self.differences.append([])
            for _descendantIndex in range(k):
                self.differences[_ancestorIndex].append(torch.tensor(self.ancestors[_ancestorIndex]) - self.descendants[_ancestorIndex][_descendantIndex])
                self.differences_singlelist.append((torch.tensor(self.ancestors[_ancestorIndex]) - self.descendants[_ancestorIndex][_descendantIndex]).reshape(1,-1))
                
        self.descendants_singlelist = torch.cat(self.descendants_singlelist)
        self.differences_singlelist = torch.cat(self.differences_singlelist)

#This subclass inherits the PyTorch Dataset class in order to create datasets of correlated memory.
class SensoryData(Dataset):
    def __init__(self, 
                 L,      #Length of each sample
                 p,      #Number of parents
                 k,      #Number of children per parent 
                 gamma   #Overlap between parent and children (1=identical, 0=no overlap)
                ):
        super().__init__()
        c = CorrelatedPatterns(L, p, k, gamma)
        memories = c.descendants_singlelist
        
        #Grab the memories generated by CorrelatedPatterns()
        self.data = memories
        self.x = memories
        self.y = memories
        self.n_samples = memories.shape[0]
    
    #Implement necessary helper functions
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.x[index], self.y[index]

Ideas to try:
- Fast weights that modify a fixed subset of the main weights only
- Meta-meta fast weights
- Fast weights that gate plasticity instead of set weights (connect with Dendritic Gated Networks)

In [116]:
class FastWeightAE(nn.Module):
    def __init__(self, 
                 n_inputs=10,
                 n_hiddens=5,
                ):
        super().__init__()
        
        self.fast_weights = Parameter(torch.rand(n_inputs, n_inputs*n_hiddens), requires_grad=True)
        self.eweight = Parameter(torch.rand(n_hiddens, n_inputs), requires_grad=True)
        self.n_inputs = n_inputs
        self.n_hiddens = n_hiddens
        
    def forward(self, X):
        #Limitation: X can only be a single sample
        
        #Generate weights using fast weight network
        generated_weights = F.linear(X, self.fast_weights.T)
        generated_weights = generated_weights.reshape_as(self.eweight.data)
        self.eweight.data = generated_weights
        
        #Run the input through the main network
        encoded = F.linear(X, self.eweight)
        encoded = F.relu(encoded)
        
        decoded = F.linear(encoded, self.eweight.T)
        
        return decoded        

In [117]:
def train(loader, #Dataloader
          model,  #Model to be trained
          loss_function, #Loss function
          optimizer, #Optimizer
          n_epochs=100 #number of epochs
         ):
    
    #Toggle to training mode
    model.train()
    
    for epoch in range(n_epochs):
        total_loss = 0
        
        #Iterate through the DataLoader's batches
        for batch, (X, y) in enumerate(loader):
            #Get the model's prediction of the input
            predicted_y = model(X)
            
            #Calculate the loss
            loss = loss_function(predicted_y, y)
            total_loss += loss
            
            #Reset gradients
            optimizer.zero_grad()
            
            #Backpropagation
            loss.backward()
            
            #Update the optimizer
            optimizer.step()
        print(f'Average loss: {total_loss/len(loader)}')

In [118]:
loss_function = nn.MSELoss()
dataset = SensoryData(10, 5,7,0.5)
loader = DataLoader(dataset, batch_size=1)
model = FastWeightAE()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [120]:
train(loader, model, loss_function, optimizer)

Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 63227.13671875
Average loss: 

In [121]:
sample = next(iter(loader))

In [122]:
model(sample[0])

tensor([[141.4532, 191.2325, 184.4556, 153.1359, 185.6811, 181.9413, 194.4612,
         176.1580, 152.2388, 166.7613]], grad_fn=<MmBackward0>)

In [123]:
sample[0]

tensor([[1., 0., 0., 1., 1., 1., 0., 0., 0., 1.]])